In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastpCommandline
import tempfile
import os

C:\Users\ralme\anaconda3\envs\BIO\Lib\site-packages\Bio\Application\__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [3]:
def limpiar_proteina(seq: str) -> str:
    seq = seq.replace(" ", "").replace("\n", "").upper()
    validos = set("ACDEFGHIKLMNPQRSTVWYX")

    if not seq:
        raise ValueError("La secuencia está vacía.")

    if not set(seq).issubset(validos):
        raise ValueError("La secuencia contiene símbolos no válidos para proteína.")

    return seq


def extraer_hits_filtrados(record, threshold=0.001):
    resultados = []

    for aln in record.alignments:
        hsp = aln.hsps[0]  
        if hsp.expect < threshold:
            identidad = (hsp.identities / hsp.align_length) * 100

            resultados.append({
                "id": aln.hit_id,
                "len": aln.length,
                "evalue": hsp.expect,
                "identidad": identidad
            })

    return resultados


def guardar_resultados(rutasalida, resultados):
    with open(rutasalida, "w") as f:
        f.write("ID\tLongitud\tE-value\t% Identidad\n")
        for r in resultados:
            f.write(f"{r['id']}\t{r['len']}\t{r['evalue']:.3e}\t{r['identidad']:.2f}\n")

In [4]:
seq_usuario = input("Introduce una secuencia de PROTEÍNA: ")

try:
    seq_limpia = limpiar_proteina(seq_usuario)
except ValueError as e:
    print(f"Error: {e}")
    raise SystemExit

print("\nRealizando BLASTp online... (esto puede tardar un poco)")

handle = NCBIWWW.qblast(
    program="blastp",
    database="nr",
    sequence=seq_limpia
)

record = NCBIXML.read(handle)

resultados_online = extraer_hits_filtrados(record)

guardar_resultados("resultados_online_proteina.txt", resultados_online)

print("\n--- RESULTADOS ONLINE ---")
print(f"Total hits con E-value < 0.001: {len(resultados_online)}")
print("Guardado en 'resultados_online_proteina.txt'")



Realizando BLASTp online... (esto puede tardar un poco)


KeyboardInterrupt: 

In [ ]:
print("\nRealizando BLASTp LOCAL...")

BLASTP_EXE = "blastp"              
DB_LOCAL = "mi_db_prot"            

with tempfile.TemporaryDirectory() as tmpdir:
    query_path = os.path.join(tmpdir, "query.fasta")
    xml_out = os.path.join(tmpdir, "blast_local.xml")

    with open(query_path, "w") as f:
        f.write(">consulta\n")
        f.write(seq_limpia + "\n")

    blast_cline = NcbiblastpCommandline(
        cmd=BLASTP_EXE,
        query=query_path,
        db=DB_LOCAL,
        outfmt=5,
        out=xml_out,
        evalue=0.001
    )

    stdout, stderr = blast_cline()

    with open(xml_out) as f:
        record_local = NCBIXML.read(f)

    resultados_local = extraer_hits_filtrados(record_local)

    guardar_resultados("resultados_local_proteina.txt", resultados_local)

print("\n--- RESULTADOS LOCALES ---")
print(f"Total hits con E-value < 0.001: {len(resultados_local)}")
print("Guardado en 'resultados_local_proteina.txt'")
